In [22]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
#for img in glob('/content/drive/My Drive/AiSd_labs/AiSd_lab_hw_3/*'):

Mounted at /content/drive


In [23]:
import time
from math import sqrt


class DisjointSetUnion:
    def __init__(self, size):
        self.parent_ = list(range(size))
        self.ranks_ = [0] * size
        self.size = size

    def find(self, node):
        if self.parent_[node] != node:
            self.parent_[node] = self.find(self.parent_[node])
        return self.parent_[node]

    def union_sets(self, first, second):
        f_root = self.find(first)
        s_root = self.find(second)
        if f_root == s_root:
            return False
        if self.ranks_[f_root] < self.ranks_[s_root]:
            self.parent_[f_root] = s_root
        elif self.ranks_[f_root] > self.ranks_[s_root]:
            self.parent_[s_root] = f_root
        else:
            self.parent_[s_root] = f_root
            self.ranks_[f_root] += 1
        self.size -= 1
        return True

    def result(self):
        return list(map(self.find, self.parent_))


def distance_1(title1, title2, num_of_words):
    # simple vec distance
    id1 = 0
    id2 = 0
    vec = []
    while id1 < len(title1) and id2 < len(title2):
        x = y = 0
        word1 = title1[id1][0]
        word2 = title2[id2][0]
        if word1 <= word2:
            x = title1[id1][1]
            id1 += 1
        if word1 >= word2:
            y = title2[id2][1]
            id2 += 1
        vec.append(abs(x - y))
    while id1 < len(title1):
        vec.append(title1[id1][1])
        id1 += 1
    while id2 < len(title2):
        vec.append(title2[id2][1])
        id2 += 1
    return sqrt(sum(map(lambda x: x * x, vec)))


def distance_2(title1, title2, num_of_words):
    # manhattan vec distance
    id1 = 0
    id2 = 0
    vec = []
    while id1 < len(title1) and id2 < len(title2):
        x = y = 0
        word1 = title1[id1][0]
        word2 = title2[id2][0]
        if word1 <= word2:
            x = title1[id1][1]
            id1 += 1
        if word1 >= word2:
            y = title2[id2][1]
            id2 += 1
        vec.append(abs(x - y))
    while id1 < len(title1):
        vec.append(title1[id1][1])
        id1 += 1
    while id2 < len(title2):
        vec.append(title2[id2][1])
        id2 += 1
    return sum(vec)


def distance_3(title1, title2, num_of_words):
    # Pearson
    x = [0] * num_of_words
    for elem in title1:
        x[elem[0] - 1] += elem[1]
    y = [0] * num_of_words
    for elem in title2:
        y[elem[0] - 1] += elem[1]
    x_m = sum(x) / len(x)
    y_m = sum(y) / len(y)
    x = [x[_] - x_m for _ in range(len(x))]
    y = [y[_] - y_m for _ in range(len(x))]
    a = sum([x[_] * y[_] for _ in range(len(x))])
    b = sum([x[_] * x[_] for _ in range(len(x))])
    c = sum([y[_] * y[_] for _ in range(len(x))])
    if a == 0:
        return 1
    return 1 - (a / sqrt(b * c))


def cluster(ver, eds, num_of_clusters):
    # return DisjointSetUnion of edges
    res = DisjointSetUnion(len(ver))
    # Your code here
    eds.sort()
    i = 0;
    while (res.size != num_of_clusters):
        res.union_sets(eds[i][1], eds[i][2])
        i += 1
    return res


t_start = time.time()

# input file
filename = 'test_50.txt'

# function of distance
distance = distance_1

# clusters num
clusters_num = 5

# word freq param
min_count = 2
max_count = 10

vertices = []
with open('/content/drive/My Drive/AiSd_labs/AiSd_lab_hw_3/task/' + filename) as f:
    doc_num = int(f.readline())
    words_num = int(f.readline())
    lines_num = int(f.readline())
    words_count = [0] * words_num
    vertices = [[] for _ in range(doc_num)]
    for i in range(lines_num):
        docID, wordID, count = map(int, f.readline().split())
        words_count[wordID - 1] += count
        if count <= max_count:
            vertices[docID - 1].append([wordID, count])

new_v = []
for elem in vertices:
    new_v.append([])
    for k in elem:
        if min_count <= words_count[k[0] - 1] <= max_count:
            new_v[-1].append(k)
    if len(new_v[-1]) == 0:
        print("Warning: vertex without words")
vertices = new_v
print('Vertices:', len(vertices))

edges = [(distance(vertices[i], vertices[j], words_num), i, j) for i in range(len(vertices) - 1) for j in range(i + 1, len(vertices))]
print('Edges:', len(edges))

components = cluster(vertices, edges, clusters_num)

comp = components.result()
d = dict()
for i in range(len(comp)):
    if comp[i] not in d:
        d[comp[i]] = [i + 1]
    else:
        d[comp[i]].append(i + 1)
for elem in d:
    print("Cluster", elem, ":", len(d[elem]), "elems", ":", d[elem])

print("Time:", time.time() - t_start)


Vertices: 50
Edges: 1225
Cluster 3 : 45 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Time: 0.13469624519348145


In [0]:
def main_part_1(files, distance, clusters_num, min_count, max_count):
    t_start = time.time()
    for filename in files:

        vertices = []
        with open('/content/drive/My Drive/AiSd_labs/AiSd_lab_hw_3/task/' + filename) as f:
            doc_num = int(f.readline())
            words_num = int(f.readline())
            lines_num = int(f.readline())
            words_count = [0] * words_num
            vertices = [[] for _ in range(doc_num)]
            for i in range(lines_num):
                docID, wordID, count = map(int, f.readline().split())
                words_count[wordID - 1] += count
                if count <= max_count:
                    vertices[docID - 1].append([wordID, count])

        new_v = []
        for elem in vertices:
            new_v.append([])
            for k in elem:
                if min_count <= words_count[k[0] - 1] <= max_count:
                    new_v[-1].append(k)
            if len(new_v[-1]) == 0:
                print("Warning: vertex without words")
        vertices = new_v
        print('Vertices:', len(vertices))

        edges = [(distance(vertices[i], vertices[j], words_num), i, j) for i in range(len(vertices) - 1) for j in range(i + 1, len(vertices))]
        print('Edges:', len(edges))

        components = cluster(vertices, edges, clusters_num)

        comp = components.result()
        d = dict()
        for i in range(len(comp)):
            if comp[i] not in d:
                d[comp[i]] = [i + 1]
            else:
                d[comp[i]].append(i + 1)
        for elem in d:
            print("Cluster", elem, ":", len(d[elem]), "elems", ":", d[elem])
        print("Time:", time.time() - t_start)
        print("--------------")

    print("Full Time:", time.time() - t_start)
    
    return 0

In [25]:
files = ['test_50.txt', 'test_400.txt', 'test_1000.txt']
main_part_1(files, distance_1, 5, 2, 10)

Vertices: 50
Edges: 1225
Cluster 3 : 45 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Time: 0.13358402252197266
--------------
Vertices: 400
Edges: 79800
Cluster 29 : 395 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140

0

In [26]:
files = ['test_50.txt', 'test_400.txt', 'test_1000.txt']
main_part_1(files, distance_2, 5, 2, 10)

Vertices: 50
Edges: 1225
Cluster 3 : 45 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Time: 0.11582827568054199
--------------
Vertices: 400
Edges: 79800
Cluster 29 : 391 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139

0

In [27]:
files = ['test_50.txt', 'test_400.txt']
main_part_1(files, distance_3, 5, 2, 10)

Vertices: 50
Edges: 1225
Cluster 5 : 45 elems : [1, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50]
Cluster 1 : 1 elems : [2]
Cluster 3 : 2 elems : [4, 5]
Cluster 37 : 1 elems : [38]
Cluster 46 : 1 elems : [47]
Time: 2.857956886291504
--------------
Vertices: 400
Edges: 79800
Cluster 111 : 392 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 14

0

In [0]:
import random

def cluster_2(ver, eds, num_of_clusters, distance, words_num):
    # return DisjointSetUnion of edges
    res = DisjointSetUnion(len(ver))
    
    clusters_leads = set()

    random_vert = random.randint(0, len(ver) - 1)
    clusters_leads.add(random_vert)
    
    while (len(clusters_leads) != num_of_clusters):

        temp_maximum_dist = -1.0
        temp_maximum_vert = -1
        
        for vert in range(0, len(ver)):
            if (vert not in clusters_leads):
                temp_dists = []
                
                for lead in clusters_leads:
                    temp_dist = distance(ver[lead], ver[vert], words_num)
                    temp_dists.append([temp_dist, vert])
                temp_dists.sort()
                
                if (temp_maximum_dist == -1):
                    temp_maximum_dist = temp_dists[0][0]
                    temp_maximum_vert = temp_dists[0][1]
                else:
                    if (temp_dists[0][0] > temp_maximum_dist):
                        temp_maximum_dist = temp_dists[0][0]
                        temp_maximum_vert = temp_dists[0][1]
                       
        clusters_leads.add(temp_maximum_vert)
        
    print('clusters_leads', clusters_leads)
  
    for vert in range(0, len(ver)):
        if (vert not in clusters_leads):
            temp_dists = []
     
            for lead in clusters_leads:
                temp_dist = distance(ver[lead], ver[vert], words_num)
                temp_dists.append([temp_dist, lead])
            temp_dists.sort()
            
            res.union_sets(temp_dists[0][1], vert)
        
    return res
                

def main_part_2(files, distance, clusters_num, min_count, max_count):
    t_start = time.time()
    
    for filename in files:

        vertices = []
        with open('/content/drive/My Drive/AiSd_labs/AiSd_lab_hw_3/task/' + filename) as f:
            doc_num = int(f.readline())
            words_num = int(f.readline())
            lines_num = int(f.readline())
            words_count = [0] * words_num
            vertices = [[] for _ in range(doc_num)]
            for i in range(lines_num):
                docID, wordID, count = map(int, f.readline().split())
                words_count[wordID - 1] += count
                if count <= max_count:
                    vertices[docID - 1].append([wordID, count])

        new_v = []
        for elem in vertices:
            new_v.append([])
            for k in elem:
                if min_count <= words_count[k[0] - 1] <= max_count:
                    new_v[-1].append(k)
            if len(new_v[-1]) == 0:
                print("Warning: vertex without words")
        vertices = new_v
        print('Vertices:', len(vertices))

        edges = [(distance(vertices[i], vertices[j], words_num), i, j) for i in range(len(vertices) - 1) for j in range(i + 1, len(vertices))]
        print('Edges:', len(edges))
        
        
        components = cluster_2(vertices, edges, clusters_num, distance, words_num)

        comp = components.result()
        d = dict()
        for i in range(len(comp)):
            if comp[i] not in d:
                d[comp[i]] = [i + 1]
            else:
                d[comp[i]].append(i + 1)
        for elem in d:
            print("Cluster", elem, ":", len(d[elem]), "elems", ":", d[elem])
        print("Time:", time.time() - t_start)
        print("--------------")

    print("Full Time:", time.time() - t_start)

    return 0

In [0]:
#files = ['test_50.txt', 'test_400.txt', 'test_1000.txt']
#main_part_2(files, distance_1, 5, 2, 10)

In [30]:
files = ['test_50.txt', 'test_400.txt', 'test_1000.txt']
main_part_2(files, distance_1, 5, 2, 10)

Vertices: 50
Edges: 1225
clusters_leads {5, 39, 42, 47, 21}
Cluster 42 : 45 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Time: 0.21822357177734375
--------------
Vertices: 400
Edges: 79800
clusters_leads {295, 8, 113, 115, 57}
Cluster 8 : 395 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 125, 

0

In [31]:
files = ['test_50.txt', 'test_400.txt', 'test_1000.txt']
main_part_2(files, distance_2, 5, 2, 10)

Vertices: 50
Edges: 1225
clusters_leads {5, 39, 47, 21, 29}
Cluster 29 : 45 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Time: 0.18192529678344727
--------------
Vertices: 400
Edges: 79800
clusters_leads {162, 3, 339, 115, 57}
Cluster 3 : 391 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124, 

0

In [32]:
files = ['test_50.txt', 'test_400.txt']
main_part_2(files, distance_3, 5, 2, 10)

Vertices: 50
Edges: 1225
clusters_leads {33, 36, 7, 29, 30}
Cluster 36 : 10 elems : [1, 4, 5, 7, 21, 33, 37, 42, 47, 48]
Cluster 33 : 7 elems : [2, 3, 20, 34, 40, 43, 49]
Cluster 30 : 17 elems : [6, 10, 11, 12, 14, 15, 16, 17, 18, 22, 24, 25, 26, 31, 32, 36, 39]
Cluster 7 : 8 elems : [8, 9, 23, 29, 38, 41, 44, 50]
Cluster 29 : 8 elems : [13, 19, 27, 28, 30, 35, 45, 46]
Time: 4.445359945297241
--------------
Vertices: 400
Edges: 79800
clusters_leads {353, 388, 271, 308, 152}
Cluster 388 : 87 elems : [1, 6, 15, 25, 27, 30, 31, 33, 36, 37, 41, 44, 46, 48, 49, 63, 64, 65, 69, 70, 78, 81, 82, 83, 87, 88, 95, 96, 106, 107, 112, 115, 118, 122, 124, 135, 136, 138, 149, 150, 151, 156, 157, 159, 160, 163, 164, 168, 170, 171, 177, 180, 185, 189, 193, 196, 201, 204, 206, 212, 218, 227, 228, 230, 241, 244, 258, 263, 267, 282, 283, 291, 295, 298, 305, 306, 319, 324, 327, 366, 370, 378, 382, 388, 389, 396, 398]
Cluster 353 : 54 elems : [2, 4, 20, 54, 56, 58, 67, 85, 90, 109, 113, 123, 134, 147, 152, 

0

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')
#files = ['test_50.txt']
#for i in range(1, 20):
#    for j in range(1, 20):
#        print(i, j)
#        main_part_1(files, distance_1, 5, i, j)
        
#main_part_1(files, distance_1, 5, 17, 36)
#main_part_1(files, distance_1, 5, 15, 60)